Workflow to take a cleaned mesh and generate a discrete mesh

In [10]:
# imports
import numpy as np
from PIL import Image
import cv2 as cv
from matplotlib import pyplot as plt
import open3d as o3d
import pyvista as pv

In [2]:
# read in the array and shapen to volume
rootpath = "F:/Dropbox (DarkVision)/Analysis/Personal/GM/FF/RandD/DVT664_exxon_olaf/"
filename = rootpath+"volume_export_768x768x1280_uint8_t.raw"
outname = rootpath+"processed_volume_export_768x768x1280_uint8_t.raw"
array = np.fromfile(filename, dtype= 'uint8')
vol = array.reshape(1280,768,768)
slice = vol[500,:,:]

Process the volume

In [3]:
# trim to only necassary extents
xy = np.sum(vol, axis=0)
validxy = np.where(xy >0)

z = np.sum(vol, axis=1)
validz = np.where(z >0)

vol = vol[validz[0][0]:validz[0][-1],validxy[0][0]:validxy[0][-1],min(validxy[1]):max(validxy[1])]

In [5]:
# preprocess
for i in range(vol.shape[0]):
    slice = vol[i,:,:]
    result = cv.fastNlMeansDenoising(slice,h=20,templateWindowSize=10,searchWindowSize=21)
    result = cv.threshold(result,80,255,cv.THRESH_BINARY)[1]
    thinned = cv.ximgproc.thinning(result)
    vol[i,:,:] = thinned

In [6]:
print(vol.shape)
vol.astype('int8').tofile(outname)

(1016, 478, 427)


convert to point cloud and save a copy

In [11]:
values = np.where(vol>=100, vol, 0)

grid = pv.UniformGrid()
grid.dimensions = np.array(values.shape) + 1

grid.origin = (0, 0, 0)  # The bottom left corner of the data set
grid.spacing = (0.3125,0.49479, 0.49479)  # These are the cell sizes along each axis

# Add the data values to the cell data
grid.cell_data["values"] = values.flatten(order="F")  # Flatten the array!

threshed = grid.threshold(100)

points = threshed.points        
np.savetxt(rootpath+"filtered_file.xyz", points)

Compute normals and load it all into Open3D data type

In [12]:
def radial_norms(points, inward=False):
    i = -1 if inward else 1
    xmean = points[:, 0].mean()
    ymean = points[:, 1].mean()
    zmean = points[:, 2].mean()
    dist = points[:, 2].max() - points[:, 2].min()
    normals = []
    for x, y, z in points:
        azimuth = np.arctan2((x-xmean), (y-ymean))
        dip = np.arcsinh((z-zmean)/dist)
        normals.append([i*np.sin(azimuth), i*np.cos(azimuth), dip])
    return np.array(normals)

normals = radial_norms(points)

In [13]:
pcd = o3d.geometry.PointCloud() # create an empty poun cloud object

pcd.points = o3d.utility.Vector3dVector(points) # feed the xyz coordinates

pcd.colors = o3d.utility.Vector3dVector(plt.get_cmap('hsv')((points[:, 2] - points[:, 2].min()) / (points[:, 2].max() - points[:, 2].min()))[:, :3]) # feed the rgb colors

pcd.normals = o3d.utility.Vector3dVector(normals) # feed the normal vectors

#o3d.visualization.draw_geometries([pcd], point_show_normal=True) # display the pcd

Create the mesh with poisson

In [15]:
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=10, scale=1.1, linear_fit=False) # running PSR algorithm
#  visualize mesh vertrex densities
densities = np.asarray(densities)
density_mesh = o3d.geometry.TriangleMesh()
density_mesh.vertices = mesh.vertices
density_mesh.triangles = mesh.triangles
density_mesh.triangle_normals = mesh.triangle_normals
density_mesh.vertex_colors = o3d.utility.Vector3dVector(plt.get_cmap('plasma')((densities - densities.min()) / (densities.max() - densities.min()))[:, :3])

In [16]:
# removing low density vertices
vertices_to_remove = densities < np.quantile(densities, 0.05)
mesh.remove_vertices_by_mask(vertices_to_remove)
#o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)
o3d.io.write_triangle_mesh("03d_mesh.ply", mesh)

[Open3D WARNING] Write Ply clamped color value to valid range


True

In [18]:
data = pv.read(rootpath+'03d_mesh.ply')
data.save(rootpath+'03d_mesh.ply')